<a href="https://www.kaggle.com/code/shubhamchand51/ml-spaceship-titanic?scriptVersionId=104780412" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Extract data
data_train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
data_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
data_sub = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

display(data_test.head(5))
display(data_test.shape)
display(data_sub.head(5))
display(data_sub.shape)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


(4277, 13)

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


(4277, 2)

# EDA

In [3]:
# Checking basic details
print(data_train.shape)
print(data_train.info())
display(data_train.describe())

(8693, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [4]:
# Checking sample data
display(data_train.head(5))

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
# Checking unique values
for i in data_train.columns:
    print(i, " = ", data_train[i].unique())

PassengerId  =  ['0001_01' '0002_01' '0003_01' ... '9279_01' '9280_01' '9280_02']
HomePlanet  =  ['Europa' 'Earth' 'Mars' nan]
CryoSleep  =  [False True nan]
Cabin  =  ['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
Destination  =  ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
Age  =  [39. 24. 58. 33. 16. 44. 26. 28. 35. 14. 34. 45. 32. 48. 31. 27.  0.  1.
 49. 29. 10.  7. 21. 62. 15. 43. 47.  2. 20. 23. 30. 17. 55.  4. 19. 56.
 nan 25. 38. 36. 22. 18. 42. 37. 13.  8. 40.  3. 54.  9.  6. 64. 67. 61.
 50. 41. 57. 11. 52. 51. 46. 60. 63. 59.  5. 79. 68. 74. 12. 53. 65. 71.
 75. 70. 76. 78. 73. 66. 69. 72. 77.]
VIP  =  [False True nan]
RoomService  =  [   0.  109.   43. ... 1569. 8586.  745.]
FoodCourt  =  [   0.    9. 3576. ... 3208. 6819. 4688.]
ShoppingMall  =  [   0.   25.  371. ... 1085.  510. 1872.]
Spa  =  [   0.  549. 6715. ... 2868. 1107. 1643.]
VRDeck  =  [   0.   44.   49. ... 1164.  971. 3235.]
Name  =  ['Maham Ofracculy' 'Juanna Vines' 'Altark Susent' ... 'F

# Drop useless columns

In [6]:
data_train = data_train.drop(['PassengerId', 'Name', 'Cabin'], axis=1)
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(4)
memory usage: 687.8+ KB
None


# Label Encoding

In [7]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
data_train['HomePlanet']=label.fit_transform(data_train['HomePlanet'])
data_train['CryoSleep']=label.fit_transform(data_train['CryoSleep'])
data_train['Destination']=label.fit_transform(data_train['Destination'])
data_train['VIP']=label.fit_transform(data_train['VIP'])
data_train['Transported']=label.fit_transform(data_train['Transported'])
display(data_train.head())

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,1


# Null Handling

In [8]:
print(data_train.isnull().sum())
#data_train=data_train.dropna()
#data_train=data_train.reset_index(drop=True)
data_train=data_train.fillna(data_train.mean())
print(data_train.isnull().sum())

HomePlanet        0
CryoSleep         0
Destination       0
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64


# Standard Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
data_train[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]=scale.fit_transform(data_train[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])
display(data_train.head())

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,0.709437,0,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0
1,0,0,2,-0.336717,0,-0.175364,-0.281669,-0.248968,0.211505,-0.230194,1
2,1,0,2,2.034566,1,-0.275409,1.955616,-0.290817,5.694289,-0.225782,0
3,1,0,2,0.290975,0,-0.340590,0.517406,0.330225,2.683471,-0.098708,0
4,0,0,2,-0.894666,0,0.118709,-0.243409,-0.038048,0.225732,-0.267258,1


# Model creation

In [10]:
# Split input and output
x=data_train.drop(['Transported'], axis=1)
y=data_train['Transported']

# Split train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

# Model selection
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train, y_train)

# Model prediction
y_pred=model.predict(x_test)

# Model score
from sklearn import metrics
print("Accuracy_Score", metrics.accuracy_score(y_test, y_pred))
print("R2_Score", metrics.r2_score(y_test, y_pred))
print("ROC_AUC_Score", metrics.roc_auc_score(y_test, y_pred))

Accuracy_Score 0.79700977573318
R2_Score 0.18618521877005145
ROC_AUC_Score 0.796408761407807


# Submission

In [11]:
data_test = data_test.drop(['Name', 'Cabin'], axis=1)
label = LabelEncoder()
data_test['HomePlanet']=label.fit_transform(data_test['HomePlanet'])
data_test['CryoSleep']=label.fit_transform(data_test['CryoSleep'])
data_test['Destination']=label.fit_transform(data_test['Destination'])
data_test['VIP']=label.fit_transform(data_test['VIP'])
data_test = data_test.fillna(data_test.mean())
#data_test=data_test.dropna()
#data_test=data_test.reset_index(drop=True)
scale = StandardScaler()
data_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]=scale.fit_transform(data_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

# Model selection
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x, y)

# Model prediction
data_pred=model.predict(data_test.drop(['PassengerId'], axis=1))

# Submission
data_x=pd.DataFrame(data_test['PassengerId'], columns=['PassengerId'])
data_y=pd.DataFrame(data_pred, columns=['Transported']).astype(bool)
data_final=pd.concat([data_x, data_y], axis=1)

"""data_final=pd.DataFrame({
    'a':data_test['PassengerId'],
    'b':data_pred.astype(bool)
})"""

display(data_final.isnull().sum())
display(data_final.shape)

data_final.to_csv('submission.csv', index=False)

PassengerId    0
Transported    0
dtype: int64

(4277, 2)

In [12]:
print(data_x.shape)
print(data_x.isnull().sum())
print(data_x.info())
print("***************************************")
print(data_y.shape)
print(data_y.isnull().sum())
print(data_y.info())
print("***************************************")
print(data_final.shape)
print(data_final.isnull().sum())
print(data_final.info())

(4277, 1)
PassengerId    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
dtypes: object(1)
memory usage: 33.5+ KB
None
***************************************
(4277, 1)
Transported    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Transported  4277 non-null   bool 
dtypes: bool(1)
memory usage: 4.3 KB
None
***************************************
(4277, 2)
PassengerId    0
Transported    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
 1   Transp